# LC.5: Longest Palindromic Subsequence | `Medium`

## Solution Approach
A palindrome has two halves that are mirror images. We move index by index, and expand outward from each index. If the left half and right half values are the same,
we have a palindrome.  The main difficult observation is to realize that a palindrome will consist of smaller palindromes, and if we can avoid re-calculating or re-detecting those smaller palindromes after we've done so once, then we can optimize our solution to linear time `O(n)`.  This solution is known as _Manachers_ algo.

## Steps
1. Pad the string with some arbitrary character (`!`) the reason being, we are initializing the two expansion pointers (`left` & `right`) at the same index, which works for odd length strings, but not for even length strings. To artifically make the technique workable on even-length strings, we're making the middle character a `!`.
2. Here's we're measuring if our current center has already been passed over by a previous center's expansion. If so, then it's possible that there exists a palindrome who's length extends the `best-right` index beyond our current value.
    - ```
        a b c b a x a b c b a
        0 0 2 0 0
                ^br
        0 0 2 0 0 5 0 0 0 0 0
                            ^ = best_right (br)
        0 0 2 0 0 5 0 0 0 0 0
                        ^   ^br
                        ^
                        here
      ```
    - Whenever we start evaluating `here`, we can see that the `best-right (br)` value is already pointing to the last index in the string. This means there's a string matching our location at the _mirrored index_. So we offset our `center` value by that previously calculated value: `2`, and we begin expansion with this offset caked into our current answer.
3. If we're here, it means we haven't expanded this far "right" so simply initialize the radius to 1 to indicate the current character we're starting from.
4. Here's where we absorb our Manacher's optimization offset to avoid re-work.
5. Undo the last change as those changes were responsible for the breaking out of the `while` loop which means the current value do **not** yield a palindrome, but the previous values do.
6. Final book-keeping and adjusting the best answers.
7. Don't forget to remove the extra values!

In [ ]:
# Time = O(n)
# Space = O(2*n + n) ~ O(n)

class Solution:
    def longestPalindrome(self, _s: str) -> str:
        self.n = len(_s)
        s = self.pad_str(_s) # Step 1
        answer = ''
        best_right = -1
        best_center = -1
        radius = [0]*len(s)

        for center in range(len(s)):
            if center <= best_right: # Step 2
                mirror = best_center - (center - best_center)
                radius[center] = min(best_right - center, radius[mirror])
            else: # Step 3
                radius[center] = 1
            left = center - radius[center] # Step 4
            right = center + radius[center]
            while left >= 0 and right < len(s) and s[left] == s[right]:
                left -= 1
                right += 1
                radius[center] += 1
            # Step 5
            left += 1
            right -= 1
            radius[center] -= 1
            n_answer = s[left:right + 1]
            # Step 6
            if self.unpad_len(n_answer) > self.unpad_len(answer):
                answer = n_answer
            if right > best_right:
                best_right = right
                best_center = center
        return answer.replace('!', '') # Step 7

    def pad_str(self, s):
        return '!'.join(list(s))

    def unpad_len(self, s):
        if not s:
            return 0
        if s[0] == '!':
            return (len(s) - 1) / 2
        return (len(s) + 1) / 2